In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [2]:
cd drive/MyDrive/Colab\ Notebooks/scripts

/content/drive/MyDrive/Colab Notebooks/scripts


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
import os
from collections import Counter
from imblearn.over_sampling import SMOTE

In [4]:
from feature_selection import forwardFeatureSelection

from NonLinCFA import NonLinCFA
from aux_GenLinCFA import prepare_target_binary

from aux_NonLinCFA import *

In [5]:
cd ..

/content/drive/MyDrive/Colab Notebooks


### Multi task functions

In [ ]:
# for binary classification 

from sklearn.metrics import accuracy_score
def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):
    
    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)
    
    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()
    
    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LogisticRegression(max_iter = 500)
    model_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())
    
    for basin in clust_basins:
        print(basin)
        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        print(accuracy_score(targets_df_test[basin].values.ravel(), res))

In [36]:
# for linear regression 

def MTL_valid(clust_basins, df_train, df_val, targets_df_train, targets_df_val):
    
    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
      one_hot_df_train = pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])
      one_hot_df_train = one_hot_df_train.set_index(df_train.index)
      clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames], one_hot_df_train),axis=1)),axis=0)
      one_hot_df_val = pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])
      one_hot_df_val = one_hot_df_val.set_index(df_val.index)
      clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames], one_hot_df_val),axis=1)),axis=0)

    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()

    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    
    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LinearRegression()
    model_ohe.fit(clusterdf_train_withClass.values, targets_df_train_unfolded.values.ravel())

    r2_train_cluster = []
    r2_val_cluster = []
    for basin in clust_basins:
        # print(basin)
        res_val = model_ohe.predict(clusterdf_val_withClass.loc[clusterdf_val_withClass[basin]==1].values)
        res_train = model_ohe.predict(clusterdf_train_withClass.loc[clusterdf_train_withClass[basin]==1].values)
        r2_train = r2_score(targets_df_train[basin].values.ravel(), res_train)
        r2_val = r2_score(targets_df_val[basin].values.ravel(), res_val)
        r2_train_cluster.append(r2_train)
        r2_val_cluster.append(r2_val)
        # print(r2_train, " ", r2_val)
    return r2_train_cluster, r2_val_cluster

In [7]:
# for linear regression 

def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):
    
    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)
    
    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()
    
    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    r2_trainval_cluster = []
    r2_test_cluster = []
    r2_val_cluster = []

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_test = LinearRegression()
    model_test.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())
    model_val = LinearRegression()
    model_val.fit(clusterdf_train_withClass.values, targets_df_train_unfolded.values.ravel())

    for basin in clust_basins:
      print(basin)
      res_test = model_test.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
      res_trainval = model_test.predict(pd.concat([clusterdf_train_withClass.loc[clusterdf_train_withClass[basin]==1], 
                                                   clusterdf_val_withClass.loc[clusterdf_val_withClass[basin]==1]]).values)
      res_val = model_val.predict(clusterdf_val_withClass.loc[clusterdf_val_withClass[basin]==1].values)
      r2_trainval = r2_score(pd.concat([targets_df_train[basin], targets_df_val[basin]], axis =0).values.ravel(), 
                             res_trainval)
      r2_trainval_cluster.append(r2_trainval)
      print("Score in trainval: ", round(r2_trainval,2))
      r2_test = r2_score(targets_df_test[basin].values.ravel(), res_test)
      r2_test_cluster.append(r2_test)
      print("Score in test: ", round(r2_test,2))
      r2_val = r2_score(targets_df_val[basin].values.ravel(), res_val)
      r2_val_cluster.append(r2_val)
      print("Score in val: ", round(r2_val,2))
    return r2_trainval_cluster, r2_test_cluster, r2_val_cluster

#Shuffle choice linear regression

In [8]:
def compute_scores_best_shuffles(shuffle_path, normal_path, basins, singleTask, best_shuffles):

  targets_df_trainVal = pd.DataFrame()
  targets_df_test = pd.DataFrame()
  targets_df_train = pd.DataFrame()
  targets_df_val = pd.DataFrame()
  best5_CMI_fulldf_train = pd.DataFrame()
  best5_CMI_fulldf_val = pd.DataFrame()
  best5_CMI_fulldf_test = pd.DataFrame()

  for basin in basins: 
    path_targets = "./csv/"
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                    max_test='2020-01-01', path=path_targets+basin+'.csv', window_size = 1)
    targets_df_train[basin] = target_df_train.mean_std
    targets_df_val[basin] = target_df_val.mean_std
    targets_df_trainVal[basin] = target_df_trainVal.mean_std
    targets_df_test[basin] = target_df_test.mean_std

    best_shuffle_id = best_shuffles[basin]

    if (best_shuffle_id != 5):
      train_temp = pd.read_csv(shuffle_path+basin+'_' + str(best_shuffle_id) + '_nonLinCFA_best5_CMI_train.csv')
      val_temp = pd.read_csv(shuffle_path+basin+'_' + str(best_shuffle_id) + '_nonLinCFA_best5_CMI_val.csv')
      test_temp = pd.read_csv(shuffle_path+basin+'_' + str(best_shuffle_id) + '_nonLinCFA_best5_CMI_test.csv')
    else:
      train_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_train.csv')
      val_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_val.csv')
      test_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_test.csv')

    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp
    
  if singleTask:
    best5_CMI_train_val = pd.concat([train_temp,val_temp])
    lin_regr_test = LinearRegression()
    lin_regr_test.fit(best5_CMI_train_val, targets_df_trainVal[basin])

    lin_regr_val = LinearRegression()
    lin_regr_val.fit(train_temp, targets_df_train[basin])
    print(basin)
    r2_trainval = lin_regr_test.score(best5_CMI_train_val, targets_df_trainVal[basin])
    print("Score in trainval: ", round(r2_trainval,2))
    r2_test = lin_regr_test.score(test_temp, targets_df_test[basin])
    print("Score in test: ", round(r2_test,2))
    r2_val = lin_regr_val.score(val_temp, targets_df_val[basin])
    print("Score in val: ", round(r2_val,2))

  else:
    print("\n---- MultiTask regression scores: ----\n")
    MTL_scores(clust_basins=basins, df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, 
               targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

## Cross-validation for time series to choose the best shuffle 3 splits fixed training set size

In [44]:
from sklearn.model_selection import TimeSeriesSplit

def shuffle_choice_cross_valid(basins, shuffle_path, normal_path):
  best_shuffles = {}
  mean_r2_shuffles = []
  for i in range(0,6):
    ### CMI best5 features
    best5_CMI_fulldf_train = pd.DataFrame()
    best5_CMI_fulldf_val = pd.DataFrame()
    targets_df_trainVal = pd.DataFrame()
    print("Cross validation for time series for shuffle ", str(i))
    for basin in basins :
      if i!=5:
        train_temp = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_train.csv')
        val_temp = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_val.csv')
      else:
        print("Cross validation for time series of ", basin, " without shuffle")
        train_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_train.csv')
        val_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_val.csv')
      
      best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
      best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
  
      # continuous target
      path_targets = "./csv/"
      target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                      max_test='2020-01-01', path=path_targets+basin+'.csv', window_size = 1)
      targets_df_trainVal[basin] = target_df_trainVal.mean_std

    X = pd.concat([best5_CMI_fulldf_train, best5_CMI_fulldf_val])
    X.reset_index(inplace=True, drop=True)
    y = targets_df_trainVal
    y.reset_index(inplace=True, drop=True)
    r2 = []
    # num of 8 days in 2 years are around 92
    tscv = TimeSeriesSplit(max_train_size=None, n_splits=3, test_size = 92)
    for train_index, valid_index in tscv.split(X):
      if (valid_index[-1] != len(X)-1):
        train_index = np.concatenate((train_index, np.array(range(valid_index[-1] + 1, len(X)))))
      print("Train length : ", len(train_index), ", validation length : ", len(valid_index)) 
      X_train, X_valid = X.iloc[train_index,:], X.iloc[valid_index,:]
      y_train, y_valid = y.iloc[train_index,:], y.iloc[valid_index,:]
      
      r2_train, r2_val = MTL_valid(basins, X_train, X_valid, y_train, y_valid)
      r2.append(r2_val)
      for idx, basin in enumerate(basins):
        # print("Train R2 MTL linear regression CMI best 5 for ", basin, " : ", round(r2_train[idx],3))
        print("Valid R2 MTL linear regression CMI best 5 for ", basin, " : ", round(r2_val[idx],3))

    mean_r2_shuffles.append(np.average(r2, axis = 0))

    best_shuffle_id = np.average(mean_r2_shuffles, axis = 1).argmax()

    for idx, basin in enumerate(basins):
      if i!=5:
        print("Mean R2 for basin ", basin ," for shuffle ", str(i), " : ", str(round(mean_r2_shuffles[i][idx], 3)))
      else:
        print("Mean R2 for basin ", basin ," without shuffle: ", str(round(mean_r2_shuffles[i][idx], 3)))
      best_shuffles[basin] = best_shuffle_id
      
    print("Mean R2 for the cluster: ", str(round(np.average(mean_r2_shuffles[i]), 3)))
    print("\n")
  # take as best shuffle the one with the highest score averaging scores
  
  if best_shuffle_id != 5:
    print("--- BEST SHUFFLE FOR THE CLUSTER : ", str(best_shuffle_id), "---\n")
  else:
    print("--- BEST SHUFFLE FOR THE CLUSTER : no shuffle---\n")

  return best_shuffles

In [45]:
features_path = './NonLinCFA/temp_prec_shuffle/'
basins = ['Emiliani1','Emiliani2','Garda_Mincio']
clusters = [['Emiliani1', 'Emiliani2', 'Garda_Mincio']]#, ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'],
          #  ['Dora','Piemonte_Sud', 'Piemonte_Nord']]
            
best_shuffles = shuffle_choice_cross_valid(basins, './NonLinCFA/temp_prec_shuffle/', './NonLinCFA/temp_prec/')

Cross validation for time series for shuffle  0
Train length :  547 , validation length :  92
Valid R2 MTL linear regression CMI best 5 for  Emiliani1  :  0.016
Valid R2 MTL linear regression CMI best 5 for  Emiliani2  :  -0.106
Valid R2 MTL linear regression CMI best 5 for  Garda_Mincio  :  0.084
Train length :  547 , validation length :  92
Valid R2 MTL linear regression CMI best 5 for  Emiliani1  :  0.365
Valid R2 MTL linear regression CMI best 5 for  Emiliani2  :  0.333
Valid R2 MTL linear regression CMI best 5 for  Garda_Mincio  :  0.298
Train length :  547 , validation length :  92
Valid R2 MTL linear regression CMI best 5 for  Emiliani1  :  0.315
Valid R2 MTL linear regression CMI best 5 for  Emiliani2  :  0.289
Valid R2 MTL linear regression CMI best 5 for  Garda_Mincio  :  0.131
Mean R2 for basin  Emiliani1  for shuffle  0  :  0.232
Mean R2 for basin  Emiliani2  for shuffle  0  :  0.172
Mean R2 for basin  Garda_Mincio  for shuffle  0  :  0.171
Mean R2 for the cluster:  0.192



In [46]:
print("---- SingleTask regression scores: ----\n")
for basin in basins: 
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', [basin], True, best_shuffles)

for cluster in clusters: 
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', cluster, False, best_shuffles)

---- SingleTask regression scores: ----

Emiliani1
Score in trainval:  0.37
Score in test:  0.27
Score in val:  0.42
Emiliani2
Score in trainval:  0.21
Score in test:  0.2
Score in val:  0.28
Garda_Mincio
Score in trainval:  0.2
Score in test:  0.15
Score in val:  0.05

---- MultiTask regression scores: ----

Emiliani1
Score in trainval:  0.36
Score in test:  0.3
Score in val:  0.35
Emiliani2
Score in trainval:  0.27
Score in test:  0.22
Score in val:  0.28
Garda_Mincio
Score in trainval:  0.24
Score in test:  0.19
Score in val:  0.16


## Cross-validation for time series to choose the best shuffle (regression) 3 splits

In [47]:
from sklearn.model_selection import TimeSeriesSplit

def shuffle_choice_cross_valid(basins, shuffle_path, normal_path):
  best_shuffles = {}
  mean_r2_shuffles = []
  for i in range(0,6):
    ### CMI best5 features
    best5_CMI_fulldf_train = pd.DataFrame()
    best5_CMI_fulldf_val = pd.DataFrame()
    targets_df_trainVal = pd.DataFrame()
    print("Cross validation for time series for shuffle ", str(i))
    for basin in basins :
      if i!=5:
        train_temp = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_train.csv')
        val_temp = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_val.csv')
      else:
        print("Cross validation for time series of ", basin, " without shuffle")
        train_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_train.csv')
        val_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_val.csv')
      
      best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
      best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
  
      # continuous target
      path_targets = "./csv/"
      target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                      max_test='2020-01-01', path=path_targets+basin+'.csv', window_size = 1)
      targets_df_trainVal[basin] = target_df_trainVal.mean_std

    X = pd.concat([best5_CMI_fulldf_train, best5_CMI_fulldf_val])
    X.reset_index(inplace=True, drop=True)
    y = targets_df_trainVal
    y.reset_index(inplace=True, drop=True)
    r2 = []
    # num of 8 days in 2 years are around 92
    tscv = TimeSeriesSplit(max_train_size=None, n_splits=3, test_size = 92)
    for train_index, valid_index in tscv.split(X):
      print("Train length : ", len(train_index), ", validation length : ", len(valid_index)) 
      X_train, X_valid = X.iloc[train_index,:], X.iloc[valid_index,:]
      y_train, y_valid = y.iloc[train_index,:], y.iloc[valid_index,:]
      
      r2_train, r2_val = MTL_valid(basins, X_train, X_valid, y_train, y_valid)
      r2.append(r2_val)
      for idx, basin in enumerate(basins):
        # print("Train R2 MTL linear regression CMI best 5 for ", basin, " : ", round(r2_train[idx],3))
        print("Valid R2 MTL linear regression CMI best 5 for ", basin, " : ", round(r2_val[idx],3))

    mean_r2_shuffles.append(np.average(r2, axis = 0))

    best_shuffle_id = np.average(mean_r2_shuffles, axis = 1).argmax()

    for idx, basin in enumerate(basins):
      if i!=5:
        print("Mean R2 for basin ", basin ," for shuffle ", str(i), " : ", str(round(mean_r2_shuffles[i][idx], 3)))
      else:
        print("Mean R2 for basin ", basin ," without shuffle: ", str(round(mean_r2_shuffles[i][idx], 3)))
      best_shuffles[basin] = best_shuffle_id
      
    print("Mean R2 for the cluster: ", str(round(np.average(mean_r2_shuffles[i]), 3)))
    print("\n")
  # take as best shuffle the one with the highest score averaging scores
  
  if best_shuffle_id != 5:
    print("--- BEST SHUFFLE FOR THE CLUSTER : ", str(best_shuffle_id), "---\n")
  else:
    print("--- BEST SHUFFLE FOR THE CLUSTER : no shuffle---\n")

  return best_shuffles

In [48]:
features_path = './NonLinCFA/temp_prec_shuffle/'
basins = ['Emiliani1','Emiliani2','Garda_Mincio']
clusters = [['Emiliani1', 'Emiliani2', 'Garda_Mincio']]#, ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'],
          #  ['Dora','Piemonte_Sud', 'Piemonte_Nord']]
            
best_shuffles = shuffle_choice_cross_valid(basins, './NonLinCFA/temp_prec_shuffle/', './NonLinCFA/temp_prec/')

Cross validation for time series for shuffle  0
Train length :  363 , validation length :  92
Valid R2 MTL linear regression CMI best 5 for  Emiliani1  :  0.0
Valid R2 MTL linear regression CMI best 5 for  Emiliani2  :  -0.115
Valid R2 MTL linear regression CMI best 5 for  Garda_Mincio  :  0.067
Train length :  455 , validation length :  92
Valid R2 MTL linear regression CMI best 5 for  Emiliani1  :  0.368
Valid R2 MTL linear regression CMI best 5 for  Emiliani2  :  0.355
Valid R2 MTL linear regression CMI best 5 for  Garda_Mincio  :  0.294
Train length :  547 , validation length :  92
Valid R2 MTL linear regression CMI best 5 for  Emiliani1  :  0.315
Valid R2 MTL linear regression CMI best 5 for  Emiliani2  :  0.289
Valid R2 MTL linear regression CMI best 5 for  Garda_Mincio  :  0.131
Mean R2 for basin  Emiliani1  for shuffle  0  :  0.228
Mean R2 for basin  Emiliani2  for shuffle  0  :  0.176
Mean R2 for basin  Garda_Mincio  for shuffle  0  :  0.164
Mean R2 for the cluster:  0.189


C

In [49]:
print("---- SingleTask regression scores: ----\n")
for basin in basins: 
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', [basin], True, best_shuffles)

for cluster in clusters: 
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', cluster, False, best_shuffles)

---- SingleTask regression scores: ----

Emiliani1
Score in trainval:  0.36
Score in test:  0.24
Score in val:  0.41
Emiliani2
Score in trainval:  0.24
Score in test:  0.14
Score in val:  0.26
Garda_Mincio
Score in trainval:  0.22
Score in test:  0.16
Score in val:  0.1

---- MultiTask regression scores: ----

Emiliani1
Score in trainval:  0.39
Score in test:  0.23
Score in val:  0.39
Emiliani2
Score in trainval:  0.29
Score in test:  0.1
Score in val:  0.35
Garda_Mincio
Score in trainval:  0.27
Score in test:  0.1
Score in val:  0.19


## Cross-validation for time series to choose the best shuffle (regression) 2 splits fixed training set

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

def shuffle_choice_cross_valid(basins, path):
  best_shuffles = {}

  for basin in basins :
    mean_r2_shuffles = []
    for i in range(0,5):
      print("Cross validation for time series of ", basin, " for shuffle ", str(i))

      ### CMI best5 features
      path_features = path
      best5_CMI_fulldf_train = pd.DataFrame()
      best5_CMI_fulldf_val = pd.DataFrame()
      train_temp = pd.read_csv(path_features+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_train.csv')
      val_temp = pd.read_csv(path_features+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_val.csv')
  
      # continuous target
      path_targets = "./csv/"
      targets_df_trainVal = pd.DataFrame()
      target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                      max_test='2020-01-01', path=path_targets+basin+'.csv', window_size = 1)
      targets_df_trainVal[basin] = target_df_trainVal.mean_std

      X = pd.concat([train_temp, val_temp])
      X.reset_index(inplace=True, drop=True)
      y = targets_df_trainVal[basin]
      y.reset_index(inplace=True, drop=True)
      r2 = []
      # num of 8 days in 2 years are around 92
      tscv = TimeSeriesSplit(max_train_size=455, n_splits=2, test_size = 92)
      for train_index, valid_index in tscv.split(X):
        print("Train length : ", len(train_index), ", validation length : ", len(valid_index)) 
        X_train, X_valid = X.iloc[train_index,:], X.iloc[valid_index,:]
        y_train, y_valid = y[train_index], y[valid_index]
        lin_regr = LinearRegression()
        lin_regr.fit(X_train, y_train)
        r2.append(lin_regr.score(X_valid, y_valid))
        print("Train R2 linear regression CMI best 5: ", round(lin_regr.score(X_train, y_train),3))
        print("Valid R2 linear regression CMI best 5: ", round(lin_regr.score(X_valid, y_valid),3))

      mean_r2_shuffles.append(np.mean(r2))
      print("Mean R2 for basin ", basin ," for shuffle ", str(i), " : ", str(round(np.mean(r2), 3)), "\n")
    print("--- BEST SHUFFLE FOR ", basin, " : ", mean_r2_shuffles.index(max(mean_r2_shuffles)), "---\n")
    best_shuffles[basin] = mean_r2_shuffles.index(max(mean_r2_shuffles))
  return best_shuffles

In [ ]:
features_path = './NonLinCFA/temp_prec_shuffle/'
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
clusters = [['Emiliani1', 'Emiliani2', 'Garda_Mincio'], ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'],
            ['Dora','Piemonte_Sud', 'Piemonte_Nord']]
            
best_shuffles = shuffle_choice_cross_valid(basins, './NonLinCFA/temp_prec_shuffle/')

In [ ]:
print("---- SingleTask regression scores: ----\n")
for basin in basins: 
  print(basin, " ", compute_scores_best_shuffles(features_path, [basin], True, best_shuffles))

for cluster in clusters: 
  compute_scores_best_shuffles(features_path, cluster, False, best_shuffles)

---- SingleTask regression scores: ----

Adda   0.24
Dora   -0.43
Emiliani1   0.24
Emiliani2   0.25
Garda_Mincio   0.16
Lambro_Olona   0.16
Oglio_Iseo   0.3
Piemonte_Nord   0.08
Piemonte_Sud   0.15
Ticino   0.19

---- MultiTask regression scores: ----

Emiliani1
0.2841472049345648
Emiliani2
0.16487942225856578
Garda_Mincio
0.18754680568435722

---- MultiTask regression scores: ----

Adda
0.09114743740096265
Lambro_Olona
0.11821828823120561
Oglio_Iseo
0.13985550325649387
Ticino
0.13269585110194126

---- MultiTask regression scores: ----

Dora
-0.5682790200180696
Piemonte_Sud
-0.4125146241379818
Piemonte_Nord
-0.3318363787948513


##Fixed validation set

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

def shuffle_choice_cross_valid(basins, shuffle_path, normal_path):
  best_shuffles = {}

  for i in range(0,6):
    ### CMI best5 features
    best5_CMI_fulldf_train = pd.DataFrame()
    best5_CMI_fulldf_val = pd.DataFrame()
    targets_df_train = pd.DataFrame()
    targets_df_val = pd.DataFrame()
    print("\nCross validation for time series for shuffle ", str(i))

    for basin in basins :
      r2 = []
      if i!=5:
        train_temp = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_train.csv')
        val_temp = pd.read_csv(shuffle_path+basin+'_'+str(i)+'_nonLinCFA_best5_CMI_val.csv')
      else:
        print("Cross validation for time series of ", basin, " without shuffle")
        train_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_train.csv')
        val_temp = pd.read_csv(normal_path+basin+'_nonLinCFA_best5_CMI_val.csv')
      # continuous target
      path_targets = "./csv/"
      target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                      max_test='2020-01-01', path=path_targets+basin+'.csv', window_size = 1)
      targets_df_train[basin] = target_df_train.mean_std
      targets_df_val[basin] = target_df_val.mean_std
      best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
      best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp

    r2_train, r2_val = MTL_valid(basins, best5_CMI_fulldf_train, best5_CMI_fulldf_val, 
                                        targets_df_train, targets_df_val)
    r2.append(r2_val)
    best_shuffle_id = np.average(r2, axis = 1).argmax()
    for idx, basin in enumerate(basins):
      print("Train R2 MTL linear regression CMI best 5 for ", basin, " : ", round(r2_train[idx],3))
      print("Valid R2 MTL linear regression CMI best 5 for ", basin, " : ", round(r2_val[idx],3))
      best_shuffles[basin] = best_shuffle_id

    print("Mean R2 for the cluster: ", str(round(np.average(r2_val), 3)))

  if best_shuffle_id != 5:
    print("--- BEST SHUFFLE FOR THE CLUSTER : ", str(best_shuffle_id), "---\n")
  else:
    print("--- BEST SHUFFLE FOR THE CLUSTER : no shuffle---\n")
  
  return best_shuffles

In [ ]:
features_path = './NonLinCFA/temp_prec_shuffle/'
basins = ['Emiliani1','Emiliani2','Garda_Mincio']
clusters = [['Emiliani1', 'Emiliani2', 'Garda_Mincio']]#, ['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'],
            #['Dora','Piemonte_Sud', 'Piemonte_Nord']]
            
best_shuffles = shuffle_choice_cross_valid(basins, './NonLinCFA/temp_prec_shuffle/', './NonLinCFA/temp_prec/')


Cross validation for time series for shuffle  0
Train R2 MTL linear regression CMI best 5 for  Emiliani1  :  0.328
Valid R2 MTL linear regression CMI best 5 for  Emiliani1  :  0.389
Train R2 MTL linear regression CMI best 5 for  Emiliani2  :  0.222
Valid R2 MTL linear regression CMI best 5 for  Emiliani2  :  0.348
Train R2 MTL linear regression CMI best 5 for  Garda_Mincio  :  0.174
Valid R2 MTL linear regression CMI best 5 for  Garda_Mincio  :  0.189
Mean R2 for the cluster:  0.309

Cross validation for time series for shuffle  1
Train R2 MTL linear regression CMI best 5 for  Emiliani1  :  0.3
Valid R2 MTL linear regression CMI best 5 for  Emiliani1  :  0.283
Train R2 MTL linear regression CMI best 5 for  Emiliani2  :  0.214
Valid R2 MTL linear regression CMI best 5 for  Emiliani2  :  0.231
Train R2 MTL linear regression CMI best 5 for  Garda_Mincio  :  0.139
Valid R2 MTL linear regression CMI best 5 for  Garda_Mincio  :  0.112
Mean R2 for the cluster:  0.209

Cross validation for ti

In [ ]:
for idx, basin in enumerate(basins):
  best_shuffles[basin] = 5

In [ ]:
print("---- SingleTask regression scores: ----\n")
for basin in basins: 
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', [basin], True, best_shuffles)

for cluster in clusters: 
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', cluster, False, best_shuffles)

---- SingleTask regression scores: ----

---- SingleTask regression scores in train+val: ----
Emiliani1   0.35
---- SingleTask regression scores in test: ----
Emiliani1   0.34
---- SingleTask regression scores in train+val: ----
Emiliani2   0.23
---- SingleTask regression scores in test: ----
Emiliani2   0.23
---- SingleTask regression scores in train+val: ----
Garda_Mincio   0.22
---- SingleTask regression scores in test: ----
Garda_Mincio   0.2

---- MultiTask regression scores in validation: ----

Emiliani1
0.30919417697598184   0.2923304570611418
Emiliani2
0.2306330902953192   0.25100775419242405
Garda_Mincio
0.17203475465004392   0.1476699356883021

---- MultiTask regression scores in test: ----

Emiliani1
0.4031084483708298
Emiliani2
0.31010939452139874
Garda_Mincio
0.2850356299508544


In [ ]:
print("---- SingleTask regression scores: ----\n")
for basin in basins: 
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', [basin], True, best_shuffles)

for cluster in clusters: 
  compute_scores_best_shuffles(features_path, './NonLinCFA/temp_prec/', cluster, False, best_shuffles)

---- SingleTask regression scores: ----

---- SingleTask regression scores in train+val: ----
Emiliani1   0.36
---- SingleTask regression scores in test: ----
Emiliani1   0.24
---- SingleTask regression scores in train+val: ----
Emiliani2   0.24
---- SingleTask regression scores in test: ----
Emiliani2   0.14
---- SingleTask regression scores in train+val: ----
Garda_Mincio   0.22
---- SingleTask regression scores in test: ----
Garda_Mincio   0.16

---- MultiTask regression scores in validation: ----

Emiliani1
0.3282275100101498   0.38914054178723556
Emiliani2
0.2224514787400339   0.34807836906169287
Garda_Mincio
0.1735667405094583   0.18893063764137907

---- MultiTask regression scores in test: ----

Emiliani1
0.23394315174160019
Emiliani2
0.10141384997940328
Garda_Mincio
0.09594425634912673
